In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import optuna

In [2]:
train_data = pd.read_csv("./processed_data.csv")

In [3]:
X = train_data.drop('readmission_id', axis=1)
y = train_data['readmission_id']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# svm_poly = SVC(kernel='poly', degree=3, random_state=42)
# svm_poly.fit(X_train, y_train)
# y_pred_poly = svm_poly.predict(X_val)
# accuracy_poly = accuracy_score(y_val, y_pred_poly)
# print(f'Polynomial Kernel Accuracy: {accuracy_poly}')

Polynomial Kernel Accuracy: 0.6428972487366648


In [5]:
# svm_rbf = SVC(kernel='rbf', random_state=42)
# svm_rbf.fit(X_train, y_train)
# y_pred_rbf = svm_rbf.predict(X_val)
# accuracy_rbf = accuracy_score(y_val, y_pred_rbf)
# print(f'RBF Kernel Accuracy: {accuracy_rbf}')

RBF Kernel Accuracy: 0.6424761370016845


In [ ]:
# Define the objective function for SVM and hyperparameter tuning
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e2)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e2)

    svm = SVC(C=C, gamma=gamma, kernel='rbf', random_state=42)
    svm.fit(X_train, y_train)
    
    preds = svm.predict(X_val)
    acc = accuracy_score(y_val, preds)
    return acc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

best_params = study.best_params
print("Best hyperparameters:", best_params)

# Train SVM using the best hyperparameters
best_svm = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel='rbf', random_state=42)
best_svm.fit(X_train, y_train)

# Evaluate the best SVM model
preds = best_svm.predict(X_val)
acc = accuracy_score(y_val, preds)
print(f"Accuracy: {acc}")

In [6]:
X_test = pd.read_csv("./test_data.csv")
test_predictions_rbf = best_svm.predict(X_test)

In [8]:
df_output = pd.read_csv("../canadian-hospital-re-admittance-challenge/sample_submission.csv")
df_output["readmission_id"] = test_predictions_rbf
df_output.to_csv("submission_svm1.csv", index=False)